# Main Program

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.cisco.com/c/en/us/td/docs/switches/datacenter/aci/apic/sw/all/syslog/guide/b_ACI_System_Messages_Guide/m-aci-f{}x-messages.html"
output = pd.DataFrame()
count = 0

def f(a):
    a = a.lstrip()
    if "If you see this fault, take the following actions:" in a :
        a = a.split('If you see this fault, take the following actions:')[1].lstrip()
    return a

In [3]:
for url_num in range(10):
    #url_num = 9
    print("start at page {}".format(url_num))

    response = requests.get(url.format(url_num))
    soup = BeautifulSoup(response.text, 'html.parser')
    b = soup.find_all("section", "section")

    for i in b:
        
        data = {}
        
        Title = i.h3.text
        data['SN'] = Title.split(': ')[0] 
        data['Name'] = Title.split(':')[1]
        
        p = i.find_all('p')
        data['Serverity'] = p[0].text.split(':')[1].lstrip()
        try:
            data['Explanation'] = p[1].text.split('Explanation:')[1].lstrip()
        except:
            data['Explanation'] = 'This Fault does not have it'
        
        try: 
            flag = 0
            R = p[2].text.split('Recommended Action:')[1].lstrip()
            data['ReconmandAction'] = f(R)
            
        except:
            flag = 1
        
        if flag == 1:
            try:
                R = i.section.text.split('Recommended Action:')[1].lstrip()
                data['ReconmandAction'] = f(R)
            except:
                data['ReconmandAction'] = 'This Fault does not have it'
            
        output = output.append(data, ignore_index = True)

start at page 0
start at page 1
start at page 2
start at page 3
start at page 4
start at page 5
start at page 6
start at page 7
start at page 8
start at page 9


In [4]:
output = output.reindex(columns = ['SN', 'Name', 'Serverity', 'Explanation', 'ReconmandAction'])
output

,SN,Name,Serverity,Explanation,ReconmandAction
0,F0020,fltFabricSelectorIssuesConfig-failed,minor\n,This fault occurs when a fabric selector (port...,Look at the configuration for issues\nFor inva...
1,F0021,fltAaaRadiusProviderRadiusProvider_Inoperable,minor\n,This fault occurs when the Radius Provider is ...,Verify the IP address and path to the Radius p...
2,F0022,fltAaaLdapProviderLdapProvider_Inoperable,minor\n,This fault occurs when the LDAP provider is no...,Verify the IP address and path to the LDAP pro...
3,F0023,fltAaaTacacsPlusProviderTacacsPlusProvider_In...,minor\n,This fault occurs when the TACACS+ provider is...,Verify the IP address and path to the TACACS+ ...
4,F0048,fltStatsAExportJobIfc-export-failed,minor\n,This fault occurs due to a Stats Export Policy...,"If you see this fault, please check the Stats ..."
...,...,...,...,...,...
52462,F99883,fltTcaEqptIngrBytes1dUnicastRate,variable\n,"This fault is caused by ""Ingress Unicast Bytes...","Check the values of ""Ingress Unicast Bytes"" st..."
52463,F99884,fltTcaEqptIngrBytes1wUnicastRate,variable\n,"This fault is caused by ""Ingress Unicast Bytes...","Check the values of ""Ingress Unicast Bytes"" st..."
52464,F99885,fltTcaEqptIngrBytes1moUnicastRate,variable\n,"This fault is caused by ""Ingress Unicast Bytes...","Check the values of ""Ingress Unicast Bytes"" st..."
52465,F99886,fltTcaEqptIngrBytes1qtrUnicastRate,variable\n,"This fault is caused by ""Ingress Unicast Bytes...","Check the values of ""Ingress Unicast Bytes"" st..."


In [5]:
output.to_excel('test.xlsx')

## END